# KBO 승패예측

투수 데이터셋에서 LG, 두산, 선발, 투수 이름 검색해서 최근 5경기 뽑기

# import

In [14]:
import requests
import json
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

# KBO data Crawling

In [ ]:
start = datetime.datetime.strptime("20190101", "%Y%m%d") # ★여기만 수정★ START DATE
end = datetime.datetime.strptime("20211130", "%Y%m%d") # ★여기만 수정★ END DATE
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
game_list=[]
data = {'MIME Type': 'application/x-www-form-urlencoded; charset=UTF-8', 
        'leId': 1,
        'srId': 0,
        'date': '20201030'
        }
data2 = {'MIME Type': 'application/x-www-form-urlencoded; charset=UTF-8', 
            'leId': 1,
            'srId': 0,
            'seasonId': 2020,
            'gameId': '20201030KTOB0'
           } 
srld = {0,1,3,4,5,6,7}
for date in date_generated:
    data['date']=date.strftime("%Y%m%d")
    for i in srld:
        data['srId'] = i 
        res = requests.post('https://www.koreabaseball.com/ws/Main.asmx/GetKboGameList', data=data)
        game = res.json()['game']
        game_num = len(game)
        for i in range(0,game_num):
            if game[i]['CANCEL_SC_ID'] == "0":
                data2['srId'] = data['srId']
                data2['seasonId'] = game[i]['G_ID'][:4]
                data2['gameId'] = game[i]['G_ID']
                data_copy = data2.copy()
                game_list.append(data_copy)
                print(data2['gameId'])
                
print(len(game_list))

In [ ]:
for data in game_list:
    if data['srId']==9:
        print(data['gameId'])

In [ ]:
result = pd.DataFrame()
pitcher_data = pd.DataFrame()
for data in game_list:
    url_base_sc = 'https://www.koreabaseball.com/ws/Schedule.asmx/GetScoreBoardScroll'
    url_base_bs = 'https://www.koreabaseball.com/ws/Schedule.asmx/GetBoxScoreScroll'
    res_sc = requests.post(url_base_sc, data = data)
    res_bs = requests.post(url_base_bs, data = data)

    try:
        table1 = res_sc.json()['table1'].replace('\r\n','')
        table3 = res_sc.json()['table3'].replace('\r\n','')

        hitter = res_bs.json()['arrHitter']
        hitter_away = hitter[0]['table3'].replace('\r\n','')
        hitter_home = hitter[1]['table3'].replace('\r\n','')

        pitcher = res_bs.json()['arrPitcher']
        pitcher_away = pitcher[0]['table'].replace('\r\n','')
        pitcher_home = pitcher[1]['table'].replace('\r\n','')
    except:
        print("pass")
        continue
    
    dict1 = json.loads(table1)
    dict3 = json.loads(table3)
    dict_h1 = json.loads(hitter_away)
    dict_h2 = json.loads(hitter_home)
    dict_p1 = json.loads(pitcher_away)
    dict_p2 = json.loads(pitcher_home)

    date = res_sc.json()['G_DT']
    season = res_sc.json()['SEASON_ID']
    away_name = res_sc.json()['AWAY_NM']
    home_name = res_sc.json()['HOME_NM']
    
    pitcher_data1 = []
    pitcher_data2 = []
    away_score1 = []
    home_score1 = []
    pitcher_data1.append(date) #경기 일자
    away_score1.append(date) #경기 일자
    
    pitcher_data1.append(season)
    away_score1.append(season) #시즌
    
    pitcher_data1.append(away_name) #원정팀 이름
    pitcher_data1.append(home_name) #상대팀
    away_score1.append(away_name) #원정팀 이름
    away_score1.append(home_name) #상대팀 이름
    
    pitcher_data1.append('Away')
    away_score1.append('Away')
    
    a = dict1['rows'][0]['row'][1]['Text'].split('>')[3]
    away_score1.append(a) #원정팀 현재 기록
    b = dict1['rows'][0]['row'][0]['Text']
    pitcher_data1.append(b) #원정팀 해당경기 승/패
    away_score1.append(b) #원정팀 해당경기 승/패
    
    pitcher_data2.append(date) #경기 일자
    home_score1.append(date)
    
    pitcher_data2.append(season)
    home_score1.append(season)
    
    pitcher_data2.append(home_name) #홈팀 이름
    pitcher_data2.append(away_name) #상대팀
    home_score1.append(home_name) #홈팀 이름
    home_score1.append(away_name) #상대팀
    
    pitcher_data2.append('Home')
    home_score1.append('Home')
    
    a = dict1['rows'][1]['row'][1]['Text'].split('>')[3]
    home_score1.append(a) #홈팀 현재 기록
    b = dict1['rows'][1]['row'][0]['Text']
    pitcher_data2.append(b) #홈팀 해당경기 승/패
    home_score1.append(b) #홈팀 해당경기 승/패
    
    away_score3 = []
    home_score3 = []
    for i in range(0,4): #R, H, E, B
        a = dict3['rows'][0]['row'][i]['Text']
        away_score3.append(a) 
        b = dict3['rows'][1]['row'][i]['Text']
        home_score3.append(b)
    
    away_score_h = []
    home_score_h = []
    for i in range(0,5): #타수, 안타, 타점, 득점, 타율
        a = dict_h1['tfoot'][0]['row'][i]['Text']
        away_score_h.append(a)
        b = dict_h2['tfoot'][0]['row'][i]['Text']
        home_score_h.append(b)
    
    for i in range(0,len(dict_p1['rows'])):
        pitcher_data1_1=[]
        for j in range(0,17):
            temp = dict_p1['rows'][i]['row'][j]['Text']
            pitcher_data1_1.append(temp)
        pitcher1 = pd.DataFrame([pitcher_data1],columns = ['date','season','team','상대팀','h&a','승/패'])
        pitcher2 = pd.DataFrame([pitcher_data1_1],columns = ['선수명','등판','결과','승','패','세','이닝','타자','투구수','타수','피안타','홈런','4사구','삼진','실점','자책','평균자책점'])
        pitcher = pd.concat([pitcher1,pitcher2],axis=1)
        pitcher_data = pitcher_data.append(pitcher,ignore_index = True)
    
    for i in range(0,len(dict_p2['rows'])):
        pitcher_data2_1=[]
        for j in range(0,17):
            temp = dict_p2['rows'][i]['row'][j]['Text']
            pitcher_data2_1.append(temp)
        pitcher1 = pd.DataFrame([pitcher_data2],columns = ['date','season','team','상대팀','h&a','승/패'])
        pitcher2 = pd.DataFrame([pitcher_data2_1],columns = ['선수명','등판','결과','승','패','세','이닝','타자','투구수','타수','피안타','홈런','4사구','삼진','실점','자책','평균자책점'])
        pitcher = pd.concat([pitcher1,pitcher2],axis=1)
        pitcher_data = pitcher_data.append(pitcher,ignore_index = True)
    
    away_score_p = []
    home_score_p = []
    
    #선발 투수
    a = dict_p1['rows'][0]['row'][0]['Text']
    away_score1.append(a)
    b = dict_p2['rows'][0]['row'][0]['Text']
    home_score1.append(b)
    
    for i in range(1,12): #타수, 안타, 타점, 득점, 타율
        a = dict_p1['tfoot'][0]['row'][i]['Text']
        away_score_p.append(a)
        b = dict_p2['tfoot'][0]['row'][i]['Text']
        home_score_p.append(b)
    
    score_board1 = pd.DataFrame([away_score1,home_score1],columns = ['Date','Season','Team','상대팀','H&A','Result','Record','선발'])
    score_board3 = pd.DataFrame([away_score3,home_score3],columns = ['R','H','E','B'])
    score_board_h = pd.DataFrame([away_score_h,home_score_h],columns = ['AB','HIT','RBI','RECh','AVG'])
    score_board_p = pd.DataFrame([away_score_p,home_score_p],columns = ['IP','HN','PIT','AP','HITp','HR','BB','SO','RECp','ER','ERA'])
    score_board = pd.concat([score_board1,score_board3,score_board_h,score_board_p],axis=1) #두 테이블에서 가져온 정보 합치기
    result = result.append(score_board,ignore_index = True)
    print(data['gameId'],"completion")
print("Finish")

In [2]:
result

NameError: name 'result' is not defined

### pitcher_data
date
season
team
h&a
승/패
선수명
등판
결과
승
패
세
이닝
타자
투구수
타수
피안타
홈런
4사구
삼진
실점
자책
평균자책점

In [ ]:
pitcher_data

타수 AB
안타 HIT
타점 RBI
득점 RECh
타율 AVG
이닝 IP
타자p HN
투구수 PIT
타수 AP
피안타 HITp
홈런 HR
사사구 BB
삼진 SO
실점 RECp
자책 ER
평자 ERA

# 데이터 전처리
#### 자료구조: dataframe
#### h: hitter
#### p: pitcher

In [1]:
is_LG = result['Team']=='LG'
LG = result[is_LG].copy()

NameError: name 'result' is not defined

In [ ]:
LG.loc[LG['H&A']=='Home','H&A']=1
LG.loc[LG['H&A']=='Away','H&A']=0
LG.loc[LG['Record']=='승','Record']=1
LG.loc[LG['Record']=='패','Record']=0
LG.loc[LG['Record']=='무','Record']=2

In [ ]:
LG_sample=LG.loc[:,['H&A','Record','선발','R','H','E','B','AB','HIT','RBI','RECh','AVG','HN','PIT','AP','HITp','HR','BB','SO','RECp','ER','ERA']]
display(LG_sample)

In [ ]:
LG_sample.to_csv('LG전처리data_2.txt')

In [ ]:
result.to_csv('전처리전체.txt')
pitcher_data.to_csv('전처리투수.txt')

전처리 투수데이터

In [16]:
jw = pd.read_csv('전처리투수.txt')

In [17]:
tail5_pitcher_data = jw[(jw['등판'] == '선발')]
retdata = jw[(jw['등판'] == '선발')]
tail5_pitcher_data

,Unnamed: 0,date,season,team,상대팀,h&a,승/패,선수명,등판,결과,...,타자,투구수,타수,피안타,홈런,4사구,삼진,실점,자책,평균자책점
0,0,2019-03-12,2019,NC,롯데,Away,패,구창모,선발,패,...,18,77,15,6,2,3,3,6,5,15.00
5,5,2019-03-12,2019,롯데,NC,Home,승,레일리,선발,승,...,21,85,19,6,0,1,4,3,3,5.40
10,10,2019-03-12,2019,KT,삼성,Away,패,쿠에바스,선발,&nbsp;,...,23,78,21,9,1,1,4,6,6,12.46
17,17,2019-03-12,2019,삼성,KT,Home,승,윤성환,선발,&nbsp;,...,18,64,16,6,4,2,1,6,4,12.00
23,23,2019-03-12,2019,SK,KIA,Away,패,김광현,선발,&nbsp;,...,13,43,13,1,0,0,2,0,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21414,21414,2021-11-15,2021,KT,두산,Home,승,소형준,선발,승,...,23,91,18,3,0,5,4,0,0,0.00
21418,21418,2021-11-17,2021,KT,두산,Away,승,데스파이네,선발,승,...,21,69,19,2,0,2,4,0,0,0.00
21422,21422,2021-11-17,2021,두산,KT,Home,패,미란다,선발,패,...,19,82,17,5,1,2,6,1,1,1.80
21427,21427,2021-11-18,2021,KT,두산,Away,승,배제성,선발,승,...,20,75,19,3,0,1,6,3,3,5.40


In [18]:
index1 = tail5_pitcher_data[tail5_pitcher_data['평균자책점'] == '-'].index
tail5_pitcher_data = tail5_pitcher_data.drop(index1)
tail5_pitcher_data = tail5_pitcher_data.drop('Unnamed: 0', axis=1)
tail5_pitcher_data = tail5_pitcher_data.drop('h&a', axis=1)
tail5_pitcher_data = tail5_pitcher_data.drop('team', axis=1)
tail5_pitcher_data = tail5_pitcher_data.drop('season', axis=1)
tail5_pitcher_data = tail5_pitcher_data.drop('결과', axis=1)
tail5_pitcher_data = tail5_pitcher_data.drop('승', axis=1)
tail5_pitcher_data = tail5_pitcher_data.drop('패', axis=1)
tail5_pitcher_data = tail5_pitcher_data.drop('세', axis=1)
tail5_pitcher_data = tail5_pitcher_data.drop('이닝', axis=1)
tail5_pitcher_data['평균자책점'] =pd.to_numeric(tail5_pitcher_data['평균자책점'])
tail5_pitcher_data.reset_index(inplace = True)

index1 = retdata[retdata['평균자책점'] == '-'].index
retdata = retdata.drop(index1)
retdata = retdata.drop('Unnamed: 0', axis=1)
retdata = retdata.drop('h&a', axis=1)
retdata = retdata.drop('team', axis=1)
retdata = retdata.drop('season', axis=1)
retdata = retdata.drop('결과', axis=1)
retdata = retdata.drop('승', axis=1)
retdata = retdata.drop('패', axis=1)
retdata = retdata.drop('세', axis=1)
retdata = retdata.drop('이닝', axis=1)
retdata['평균자책점'] =pd.to_numeric(retdata['평균자책점'])
retdata.reset_index(inplace = True)

In [19]:
def jww(data, linenumber):
  agteam = data.iloc[linenumber,2]
  pitcher = data.iloc[linenumber,4]
  datedata = data.loc[0:linenumber-1]
  newdata = datedata[(datedata['선수명'] == pitcher)].tail()

  if(len(newdata)<5):
    return(retdata)

  retdata.loc[linenumber, '선수명'] = pitcher
  retdata.loc[linenumber,'상대팀'] = agteam
  retdata.loc[linenumber,'승/패'] = data.iloc[linenumber, 3]
  retdata.loc[linenumber,'투구수'] = newdata['투구수'].mean()
  retdata.loc[linenumber,'타수'] = newdata['타수'].mean()
  retdata.loc[linenumber,'피안타'] = newdata['피안타'].mean()
  retdata.loc[linenumber,'홈런'] = newdata['홈런'].mean()
  retdata.loc[linenumber,'4사구'] = newdata['4사구'].mean()
  retdata.loc[linenumber,'삼진'] = newdata['삼진'].mean()
  retdata.loc[linenumber,'실점'] = newdata['실점'].mean()
  retdata.loc[linenumber,'자책'] = newdata['자책'].mean()
  retdata.loc[linenumber,'평균자책점'] = newdata['평균자책점'].mean()
 
  return (retdata)

In [20]:
for i in range (0, 4603):
  jww(tail5_pitcher_data, i)

In [22]:
retdata[retdata['선수명']=='윤성환']

,index,date,상대팀,승/패,선수명,등판,타자,투구수,타수,피안타,홈런,4사구,삼진,실점,자책,평균자책점
3,17,2019-03-12,KT,승,윤성환,선발,18,64.0,16.0,6.0,4.0,2.0,1.0,6.0,4.0,12.000
60,336,2019-03-19,롯데,패,윤성환,선발,23,59.0,20.0,5.0,0.0,2.0,0.0,3.0,2.0,6.750
206,986,2019-04-07,SK,패,윤성환,선발,23,86.0,21.0,4.0,1.0,2.0,3.0,1.0,1.0,1.500
249,1160,2019-04-13,KT,패,윤성환,선발,22,80.0,21.0,8.0,0.0,1.0,2.0,3.0,3.0,3.270
302,1416,2019-04-19,한화,패,윤성환,선발,22,80.0,16.0,4.0,0.0,3.0,4.0,2.0,2.0,3.180
341,1600,2019-04-25,SK,패,윤성환,선발,28,73.8,18.8,5.4,1.0,2.0,2.0,3.0,2.4,5.340
386,1799,2019-05-01,KIA,승,윤성환,선발,25,80.6,20.4,5.6,0.2,2.2,2.4,2.4,2.2,3.644
445,2056,2019-05-08,NC,승,윤성환,선발,28,89.2,21.0,6.4,0.2,2.2,3.2,2.2,2.2,3.002
500,2286,2019-05-15,두산,패,윤성환,선발,26,91.8,22.4,6.0,0.0,1.8,3.4,2.0,2.0,3.238
557,2539,2019-05-22,한화,승,윤성환,선발,22,90.8,22.2,5.6,0.0,2.2,3.4,2.0,2.0,3.170


In [23]:
retdata.to_csv('전처리완료투수.txt')

타자 데이터

In [25]:
jw2 = pd.read_csv('전처리전체.txt')
jw2_filtered = jw2.loc[:, ['Date', 'Season','Team' , '상대팀', 'R', 'H', 'E', 'B', 'AB', 'HIT', 'RBI', 'AVG']]
jw2_fortest = jw2.loc[:, ['Date', 'Season','Team' , '상대팀', 'R', 'H', 'E', 'B', 'AB', 'HIT', 'RBI', 'AVG']]
jw2_forpita = jw2.loc[:, ['Date', 'Season','Team' , '상대팀', 'R', 'RECp']]
jw2_resultpita = jw2.loc[:, ['Date', 'Season','Team' , '상대팀', 'R', 'RECp']]
jw2_filtered

,Date,Season,Team,상대팀,R,H,E,B,AB,HIT,RBI,AVG
0,2019-03-12,2019,NC,롯데,4,10,1,1,34,10,4,0.294
1,2019-03-12,2019,롯데,NC,6,9,0,5,31,9,5,0.290
2,2019-03-12,2019,KT,삼성,12,11,0,8,39,11,12,0.282
3,2019-03-12,2019,삼성,KT,13,20,1,2,45,20,13,0.444
4,2019-03-12,2019,SK,KIA,1,5,3,3,30,5,1,0.167
...,...,...,...,...,...,...,...,...,...,...,...,...
4599,2021-11-15,2021,KT,두산,6,8,0,4,30,8,6,0.262
4600,2021-11-17,2021,KT,두산,3,8,0,6,30,8,3,0.264
4601,2021-11-17,2021,두산,KT,1,5,0,2,32,5,1,0.213
4602,2021-11-18,2021,KT,두산,8,13,1,8,35,13,8,0.294


In [26]:
def jww2(data, linenumber):
  agteam = data.iloc[linenumber,3]
  datedata = data.loc[0:linenumber-1]
  newdata = datedata[(datedata['상대팀'] == agteam)].tail()

  if(len(newdata)<5):
    return(jw2_fortest)

  jw2_fortest.loc[linenumber,'상대팀'] = agteam
  jw2_fortest.loc[linenumber,'R'] = newdata['R'].mean()
  jw2_fortest.loc[linenumber,'H'] = newdata['H'].mean()
  jw2_fortest.loc[linenumber,'E'] = newdata['E'].mean()
  jw2_fortest.loc[linenumber,'B'] = newdata['B'].mean()
  jw2_fortest.loc[linenumber,'HIT'] = newdata['HIT'].mean()
  jw2_fortest.loc[linenumber,'RBI'] = newdata['RBI'].mean()
  jw2_fortest.loc[linenumber,'AVG'] = newdata['AVG'].mean()
  
  return (jw2_fortest)

In [27]:
for i in range(0, 4604):
  jww2(jw2_filtered, i)

In [28]:
jw2_fortest.to_csv('타자전처리완료.txt')

In [29]:
def jww3(data, linenumber):
  agteam = data.iloc[linenumber,3]
  datepitadata = data.loc[0:linenumber-1]
  newpitadata = datepitadata[(datepitadata['상대팀'] == agteam)].tail()

  if(len(newpitadata)<5):
    Rmean = data.loc[linenumber, 'R']
    Recmean = data.loc[linenumber,'RECp']
    jw2_resultpita.loc[linenumber,'pita'] = Rmean*Rmean / (Rmean*Rmean + Recmean*Recmean)
    return(jw2_resultpita)

  Rmean = newpitadata['R'].mean()
  Recmean = newpitadata['RECp'].mean()

  jw2_resultpita.loc[linenumber,'상대팀'] = agteam
  jw2_resultpita.loc[linenumber,'R'] = Rmean
  jw2_resultpita.loc[linenumber,'RECp'] = Recmean
  jw2_resultpita.loc[linenumber,'pita'] = Rmean*Rmean / (Rmean*Rmean + Recmean*Recmean)
  
  return (jw2_resultpita)

피타고리안 승률   최근 20경기로 계산

In [30]:
for i in range (0, 4604):
  jww3(jw2_forpita, i)

In [31]:
jw2_resultpita

,Date,Season,Team,상대팀,R,RECp,pita
0,2019-03-12,2019,NC,롯데,4.0,6.0,0.307692
1,2019-03-12,2019,롯데,NC,6.0,4.0,0.692308
2,2019-03-12,2019,KT,삼성,12.0,13.0,0.460064
3,2019-03-12,2019,삼성,KT,13.0,12.0,0.539936
4,2019-03-12,2019,SK,KIA,1.0,4.0,0.058824
...,...,...,...,...,...,...,...
4599,2021-11-15,2021,KT,두산,4.6,6.4,0.340631
4600,2021-11-17,2021,KT,두산,4.0,6.0,0.307692
4601,2021-11-17,2021,두산,KT,2.0,4.2,0.184843
4602,2021-11-18,2021,KT,두산,4.0,4.2,0.475624


In [32]:
jw2_resultpita.to_csv('피타고리안.txt')